## 1. duckduckgo

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
search_tool.invoke('indian cricket news')

c:\Users\LENOVO\OneDrive\Desktop\ds\langchain\14_tools\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


"Latest and Breaking Cricket News: Check live cricket scores, schedule, results, scorecard, upcoming matches list and ball by ball commentary along with player stats, player profiles, team profiles ... Mar 9, 2025 · India topped Group A by winning all three matches and went on to beat Australia by four wickets in the semifinal. New Zealand, meanwhile, defeated South Africa by 50 runs in the other semifinal. Aug 8, 2025 · After their gripping 2-2 Test series draw against England, India’s cricket calendar remains packed for the remainder of 2025, with tournaments and bilateral series across formats. Jun 29, 2025 · On this day in 2024, Team India gave its cricket-obsessed fans a night to remember forever by lifting the T20 World Cup trophy. The victory was more than just a title. Aug 5, 2025 · India clinched a six-run victory against England in the final Test at The Oval, drawing the series 2-2. Mohammed Siraj's stellar performance led India's comeback after initial defeats."

In [3]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## 2. shell

In [7]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()
shell_tool.invoke('whoami')

Executing command:
 whoami


c:\Users\LENOVO\OneDrive\Desktop\ds\langchain\14_tools\venv\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


'yash\\lenovo\r\n'

## custom tools
### 1. using decorator

1. create a function with comment
2. add data type hints
3. add decorator on top

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(x: int, y: int) -> int:
    """multiplies two numbers."""
    return x*y

In [12]:
multiply({"x": 5, "y": 10}) 

50

every tool has 3 attributes

In [13]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiplies two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


this is what llm sees

In [14]:
multiply.args_schema.model_json_schema()

{'description': 'multiplies two numbers.',
 'properties': {'x': {'title': 'X', 'type': 'integer'},
  'y': {'title': 'Y', 'type': 'integer'}},
 'required': ['x', 'y'],
 'title': 'multiply',
 'type': 'object'}

### 2. using structuredtool

In [15]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [24]:
class MultiplyInput(BaseModel):
    a: int = Field(required = True, description="The first number to multiply")
    b: int = Field(required = True, description="The second number to multiply")

c:\Users\LENOVO\OneDrive\Desktop\ds\langchain\14_tools\venv\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
c:\Users\LENOVO\OneDrive\Desktop\ds\langchain\14_tools\venv\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [26]:
def multiply_func(a, b) -> int:
    return a * b

In [27]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="multiplies two numbers",
    args_schema=MultiplyInput  
)

In [28]:
multiply_tool.invoke({'a': 5, 'b': 10})

50

### 3. basetool class

In [29]:
from langchain_core.tools import BaseTool
from typing import Type

In [31]:
class addition(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

c:\Users\LENOVO\OneDrive\Desktop\ds\langchain\14_tools\venv\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
c:\Users\LENOVO\OneDrive\Desktop\ds\langchain\14_tools\venv\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [32]:
class add(BaseTool):
    name: str = "addition"
    description: str = "add two numbers"

    args_schema: Type[BaseModel] = addition

    def _run(self, a: int, b: int) -> int:
        return a + b

In [33]:
obj = add()
obj.invoke({'a': 5, 'b': 10})

15

## toolkits

In [34]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [35]:
class mathtoolkit:
    def get_tools(self):
        return [add, multiply]

In [37]:
toolkit = mathtoolkit()
tools = toolkit.get_tools()
for i in tools:
    print(i.name, "->", i.description)

add -> Add two numbers
multiply -> Multiply two numbers
